In [ ]:
%matplotlib inline
from __future__ import division, print_function

# OpenMM Imports
from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *
from sys import stdout
import pdbfixer
import tempfile
import pytraj as pt
import nglview as nv
from math import ceil

import parmed as pmd
import mdtraj as md

from io import StringIO
import numpy as np
import pytraj as pt

print(version.full_version)

for i in range(Platform.getNumPlatforms()):
    print(Platform.getPlatform(i).getName())
print(Platform.getPluginLoadFailures())

import os
from random import random

In [ ]:
def findForce(system, forcetype, add=True):
    """ Finds a specific force in the system force list - added if not found."""
    for force in system.getForces():
        if isinstance(force, forcetype):
            return force
    if add==True:
        system.addForce(forcetype())
        return findForce(system, forcetype)
    return None

In [ ]:
# Loads a structure file from the protein data bank
fixer = pdbfixer.PDBFixer(pdbid='2M4J')
numChains = len(list(fixer.topology.chains()))
print(numChains)
fixer.findMissingResidues()     
fixer.findMissingAtoms()
fixer.addMissingAtoms()        
fixer.addMissingHydrogens(pH=7.4)
print(len(list(fixer.topology.chains())))
PDBFile.writeFile(fixer.topology, fixer.positions, open('2M4J.pdb','w'))

In [ ]:
# Centers the structure in a cubic box 
PACKMOL_INPUT = """ 
# Mixture 

tolerance %f
output %s
add_amber_ter

structure 2M4J.pdb
  number 1 
  center
  fixed 150. 150. 150. 0. 0. 0.
end structure
"""

PACKMOL_INPUT = PACKMOL_INPUT % (1,'2M4J.pdb')
file_handle = open('packmol_input.txt', 'w')
file_handle.write(PACKMOL_INPUT)
file_handle.close()
!packmol < packmol_input.txt > /dev/null
pdb = PDBFile('2M4J.pdb')
pdb.topology.setUnitCellDimensions([30,30,30])
PDBFile.writeFile(pdb.topology, pdb.positions, open('2M4J.pdb', 'w'))

In [ ]:
# Force field and topology
forcefield = ForceField('amber99sb.xml', 'amber99_obc.xml')
system1 = forcefield.createSystem(pdb.topology, nonbondedMethod=CutoffNonPeriodic, nonbondedCutoff=1.5*nanometers)

gb_force = findForce(system1, GBSAOBCForce)

structure = pmd.openmm.topsystem.load_topology(pdb.topology, system1, pdb.positions)
structure.save('2M4J.prmtop', format='amber', overwrite=True)
structure.save('2M4J.inpcrd', format='rst7', overwrite=True)
inpcrd = AmberInpcrdFile('2M4J.inpcrd')
prmtop = AmberPrmtopFile('2M4J.prmtop')

system2 = prmtop.createSystem(nonbondedMethod=CutoffNonPeriodic, nonbondedCutoff=1.5*nanometers, soluteDielectric=2.0,
                             implicitSolvent=OBC2, implicitSolventSaltConc=0.15*molar,temperature=300*kelvin)

cgb_force = findForce(system2, CustomGBForce)

tot_charge = 0*elementary_charge
for i in range(cgb_force.getNumParticles()):
    charge = gb_force.getParticleParameters(i)[0]
    radius = gb_force.getParticleParameters(i)[1]
    scale = gb_force.getParticleParameters(i)[2]
    tot_charge = tot_charge + charge
    cgb_force.setParticleParameters(i,gb_force.getParticleParameters(i))
print(tot_charge)
parm = pmd.load_file('2M4J.prmtop', '2M4J.inpcrd')

In [ ]:
# External force squeezing the structure along the z-axis
for force in system2.getForces():
    print(force.this)
    
ce_force = CustomExternalForce("k*(z-cz)^2")
ce_force.addGlobalParameter("k", 0.02*kilocalories_per_mole/angstroms**2)
ce_force.addGlobalParameter("cz", 15*nanometers)
for i, atom_crd in enumerate(parm.positions):
    if parm.atoms[i].residue.idx in cnstr_res:
        ce_force.addParticle(i)
system2.addForce(ce_force)

for force in system2.getForces():
    print(force.this)

In [ ]:
# Short MD simulation
integrator = LangevinIntegrator(300*kelvin, 1.0/picoseconds, 1*femtoseconds)
integrator.setConstraintTolerance(0.0001)

platform = Platform.getPlatformByName('CUDA')
properties = {'CudaPrecision': 'mixed','CudaDeviceIndex': '1'}
simulation = Simulation(pdb.topology, system2, integrator, platform, properties)
simulation.context.setPositions(pdb.positions)

print(platform.getPropertyValue(simulation.context,'CudaDeviceIndex'))

cgb_force = findForce(system2, CustomGBForce)
print(gb_force.getParticleParameters(0)[0],gb_force.getParticleParameters(0)[1],gb_force.getParticleParameters(0)[2])
print(cgb_force.getParticleParameters(0)[0],cgb_force.getParticleParameters(0)[1],cgb_force.getParticleParameters(0)[2])

print('Minimizing...')
simulation.minimizeEnergy(tolerance=1)

print('Calculating energy...')
state = simulation.context.getState(getEnergy=True,enforcePeriodicBox=False)
print('Energy = %.5f' %
    (state.getPotentialEnergy().value_in_unit_system(md_unit_system)))

simulation.context.setVelocitiesToTemperature(300*kelvin)

simulation.reporters.append(app.DCDReporter('2M4J.dcd', 200))
simulation.reporters.append(app.StateDataReporter(stdout, 200, time=True, 
    potentialEnergy=True, temperature=True, progress=True, remainingTime=True,
    speed=True, totalSteps=10000, separator='\t'))

simulation.step(10000)

print('Saving pdb file')
# in the following line, the last frame is saved as a pdb
positions = simulation.context.getState(getPositions=True).getPositions()
PDBFile.writeFile(simulation.topology, positions, open('2M4J.pdb', '+w'))

print('Done!')

In [ ]:
# The obtained structure is stacked along the z-axis to form a fibril
z_pos = np.empty(0,dtype=int)
for i in range(24):
    init = 100
    z_pos = np.append(z_pos,init+i*18)
print(z_pos.tolist())

# write input file for packmol

PACKMOL_INPUT = """ 
# Mixture 

tolerance %f
output %s
add_amber_ter

structure 2M4J.pdb
  number 1 
  center
  fixed 150. 150. 100. 0. 0. 0.
end structure
structure 2M4J.pdb
  number 1
  center
  fixed 150. 150. %f 0. 0. 0.
end structure
structure 2M4J.pdb
  number 1
  center
  fixed 150. 150. %f 0. 0. 0.
end structure
structure 2M4J.pdb
  number 1
  center
  fixed 150. 150. %f 0. 0. 0.
end structure
structure 2M4J.pdb
  number 1 
  center
  fixed 150. 150. %f 0. 0. 0.
end structure
structure 2M4J.pdb
  number 1
  center
  fixed 150. 150. %f 0. 0. 0.
end structure
structure 2M4J.pdb
  number 1
  center
  fixed 150. 150. %f 0. 0. 0.
end structure
structure 2M4J.pdb
  number 1
  center
  fixed 150. 150. %f 0. 0. 0.
end structure
structure 2M4J.pdb
  number 1
  center
  fixed 150. 150. %f 0. 0. 0.
end structure
structure 2M4J.pdb
  number 1
  center
  fixed 150. 150. %f 0. 0. 0.
end structure
structure 2M4J.pdb
  number 1
  center
  fixed 150. 150. %f 0. 0. 0.
end structure
structure 2M4J.pdb
  number 1 
  center
  fixed 150. 150. %f 0. 0. 0.
end structure
structure 2M4J.pdb
  number 1
  center
  fixed 150. 150. %f 0. 0. 0.
end structure
structure 2M4J.pdb
  number 1
  center
  fixed 150. 150. %f 0. 0. 0.
end structure
structure 2M4J.pdb
  number 1
  center
  fixed 150. 150. %f 0. 0. 0.
end structure
structure 2M4J.pdb
  number 1
  center
  fixed 150. 150. %f 0. 0. 0.
end structure
structure 2M4J.pdb
  number 1
  center
  fixed 150. 150. %f 0. 0. 0.
end structure
structure 2M4J.pdb
  number 1
  center
  fixed 150. 150. %f 0. 0. 0.
end structure
structure 2M4J.pdb
  number 1 
  center
  fixed 150. 150. %f 0. 0. 0.
end structure
structure 2M4J.pdb
  number 1
  center
  fixed 150. 150. %f 0. 0. 0.
end structure
structure 2M4J.pdb
  number 1
  center
  fixed 150. 150. %f 0. 0. 0.
end structure
structure 2M4J.pdb
  number 1
  center
  fixed 150. 150. %f 0. 0. 0.
end structure


"""

PACKMOL_INPUT = PACKMOL_INPUT % (1,'2M4Jfib.pdb',z_pos[1],z_pos[2],z_pos[3],z_pos[4],z_pos[5],z_pos[6],z_pos[7],
                            z_pos[8],z_pos[9],z_pos[10],z_pos[11],z_pos[12],z_pos[13],z_pos[14],
                            z_pos[15],z_pos[16],z_pos[17],z_pos[18],z_pos[19],z_pos[20],z_pos[21] )
                                 
file_handle = open('packmol_input.txt', 'w')
file_handle.write(PACKMOL_INPUT)
file_handle.close()
!packmol < packmol_input.txt > /dev/null

pdb = PDBFile('2M4Jfib.pdb')
pdb.topology.setUnitCellDimensions([30,30,0.1*z_pos[7]+10])
PDBFile.writeFile(pdb.topology, pdb.positions, open('2M4Jfib.pdb', 'w'))

In [ ]:
# Force field and topology
pdb = PDBFile('2M4Jfib.pdb')
forcefield = ForceField('amber99sb.xml', 'amber99_obc.xml')
system1 = forcefield.createSystem(pdb.topology, nonbondedMethod=CutoffNonPeriodic, nonbondedCutoff=1.5*nanometers)

gb_force = findForce(system1, GBSAOBCForce)

structure = pmd.openmm.topsystem.load_topology(pdb.topology, system1, pdb.positions)
structure.save('2M4J.prmtop', format='amber', overwrite=True)
structure.save('2M4J.inpcrd', format='rst7', overwrite=True)
inpcrd = AmberInpcrdFile('2M4J.inpcrd')
prmtop = AmberPrmtopFile('2M4J.prmtop')

system2 = prmtop.createSystem(nonbondedMethod=CutoffNonPeriodic, nonbondedCutoff=1.5*nanometers, soluteDielectric=2.0,
                             implicitSolvent=OBC2, implicitSolventSaltConc=0.15*molar,temperature=300*kelvin)

cgb_force = findForce(system2, CustomGBForce)

tot_charge = 0*elementary_charge
for i in range(cgb_force.getNumParticles()):
    charge = gb_force.getParticleParameters(i)[0]
    radius = gb_force.getParticleParameters(i)[1]
    scale = gb_force.getParticleParameters(i)[2]
    tot_charge = tot_charge + charge
    cgb_force.setParticleParameters(i,gb_force.getParticleParameters(i))
print(tot_charge)
parm = pmd.load_file('2M4J.prmtop', '2M4J.inpcrd')

In [ ]:
# Simulated annealing and MD simulation
integrator = LangevinIntegrator(380*kelvin, 1.0/picoseconds, 2*femtoseconds)
integrator.setConstraintTolerance(0.0001)

platform = Platform.getPlatformByName('CUDA')
properties = {'CudaPrecision': 'mixed','CudaDeviceIndex': '1'}
simulation = Simulation(pdb.topology, system2, integrator, platform, properties)
simulation.context.setPositions(pdb.positions)

print(platform.getPropertyValue(simulation.context,'CudaDeviceIndex'))

cgb_force = findForce(system2, CustomGBForce)
print(gb_force.getParticleParameters(0)[0],gb_force.getParticleParameters(0)[1],gb_force.getParticleParameters(0)[2])
print(cgb_force.getParticleParameters(0)[0],cgb_force.getParticleParameters(0)[1],cgb_force.getParticleParameters(0)[2])

print('Minimizing...')
simulation.minimizeEnergy(10)

print('Calculating energy...')
state = simulation.context.getState(getEnergy=True,enforcePeriodicBox=False)
print('Energy = %.5f' %
    (state.getPotentialEnergy().value_in_unit_system(md_unit_system)))

positions = simulation.context.getState(getPositions=True).getPositions()
PDBFile.writeFile(simulation.topology, positions, open('2M4Jfib.pdb', '+w'))

for i in np.arange(0,80,1):
    integrator.setTemperature((380.-i)*kelvin)
    simulation.reporters.append(app.StateDataReporter(stdout, 2500, time=True, 
    potentialEnergy=True, temperature=True, progress=True, remainingTime=True,
    speed=True, totalSteps=10000, separator='\t'))
    simulation.step(10000)
    
positions = simulation.context.getState(getPositions=True).getPositions()
PDBFile.writeFile(simulation.topology, positions, open('2M4Jfib.pdb', '+w'))
    
for i in np.arange(0,80,1):
    integrator.setTemperature((380.-i)*kelvin)
    simulation.reporters.append(app.StateDataReporter(stdout, 2500, time=True, 
    potentialEnergy=True, temperature=True, progress=True, remainingTime=True,
    speed=True, totalSteps=10000, separator='\t'))
    simulation.step(10000)

integrator.setTemperature(300*kelvin)
simulation.reporters.append(app.DCDReporter('2M4Jfib.dcd', 1000))
simulation.reporters.append(app.StateDataReporter(stdout, 1000, time=True, 
    potentialEnergy=True, temperature=True, progress=True, remainingTime=True,
    speed=True, totalSteps=150000, separator='\t'))

simulation.step(150000)

print('Saving pdb file')
# in the following line, the last frame is saved as a pdb
positions = simulation.context.getState(getPositions=True).getPositions()
PDBFile.writeFile(simulation.topology, positions, open('2M4Jfib.pdb', '+w'))

print('Done!')